In [364]:
import requests
import ast
import pandas as pd
from rich import print as printr
import json
from pathlib import Path

In [365]:
# Define query for Fnugg API.
endpoint = 'https://api.fnugg.no/search'
parameters = {
    'size': 150,
    'sourceFields': 'name,contact,resort_open,region,location,weather_zones,conditions,last_updated,site_path,lifts,slopes',
}

In [366]:
# Access Fnug API and confirm successful GET request.
r = requests.get(endpoint, params=parameters)
print(r.status_code)
data_dict = r.json()

200


In [367]:
print ('Number of resorts matching query = ' + str(len(data_dict['hits']['hits'])))
resorts = data_dict['hits']['hits']

Number of resorts matching query = 121


In [368]:
# Save to file for futre use.
with open('fnugg-data.json', 'w') as json_file:
    json.dump(data_dict, json_file, indent=4)

In [369]:
# Reopen from file.
with open('fnugg-data.json', 'r') as json_file:
    data_dict = json.load(json_file)

In [370]:
# Define DataFrame ro capture key details from API reponse.
columns = ['name', 'region', 'resort_open', 'latitude', 'longitude', 'weather_zone_type',
        'snow_at_top_current', 'snow_at_bottom_current', 'last_updated', 'total_slopes',
        'slopes_open', 'total_lifts', 'lifts_open', 'url', 'weekend_forecast']
df = pd.DataFrame(columns=columns)

# Loop through resorts and collect data.
for resort in resorts:
    row = {
        'name': resort['_source']['name'],
        'region': resort['_source']['region'],
        'resort_open': resort['_source']['resort_open'],
        'latitude': resort['_source']['location']['lat'],
        'longitude': resort['_source']['location']['lon'],
        'weather_zone_type': resort['_source']['weather_zones']['type'],
        'snow_at_top_current': resort['_source']['conditions']['current_report']['top']['snow']['depth_slope'],
        'last_updated': resort['_source']['last_updated'],
        'total_slopes': resort['_source']['slopes']['count'],
        'slopes_open': resort['_source']['slopes']['open'],
        'total_lifts': resort['_source']['lifts']['count'],
        'lifts_open': resort['_source']['slopes']['open'],
        'weekend_forecast': resort['_source']['conditions']['forecast']['weekend']['symbol']['fnugg_id'],
        'url': 'https://fnugg.no/' + resort['_source']['site_path'],
    }
    if 'bottom' in resort['_source']['conditions']['current_report']: # Some resorts only have 'top' measurements.
        row['snow_at_bottom_current'] = resort['_source']['conditions']['current_report']['bottom']['snow']['depth_slope']
    else:
        row['snow_at_bottom_current'] = None

    df.loc[len(df)] = row


In [385]:
weather_codes = {
1: "Radiant sun",
2: "Mostly sun",
3: "Cloudy",
4: "Snow is reported here",
5: "It's snowing!",
6: "Risk of rain",
7: "It's raining",
8: "Danger of sleet",
9: "It's nonsense..",
}

In [398]:
list(weather_codes.values())

['Radiant sun',
 'Mostly sun',
 'Cloudy',
 'Snow is reported here',
 "It's snowing!",
 'Risk of rain',
 "It's raining",
 'Danger of sleet',
 "It's nonsense.."]

In [388]:
df['weekend_forecast'] = df['weekend_forecast'].map(weather_codes)

In [394]:
# Create a DataFrame for the Shiny dashboard.
renamer = {
    'name': 'Name',
    'region': 'Region',
    'resort_open': 'Resort Open',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'weather_zone_type': 'Weather Zone Type',
    'snow_at_top_current': 'Snow on Top (cm)',
    'snow_at_bottom_current': 'Snow at Bottom (cm)',
    'last_updated': 'Last Updated',
    'total_slopes': 'Total Slopes',
    'slopes_open': 'Slopes Open',
    'total_lifts': 'Total Lifts',
    'lifts_open': 'Lifts Open',
    'weekend_forecast': 'Weekend Forecast',
    'url': 'Url',
    }

df_dashboard = df.rename(columns=renamer)
df_dashboard.head()

,Name,Region,Resort Open,Latitude,Longitude,Weather Zone Type,Snow on Top (cm),Snow at Bottom (cm),Last Updated,Total Slopes,Slopes Open,Total Lifts,Lifts Open,Url,Weekend Forecast
0,Arena Overøye Stordal Alpinsenter,[Nord-Vestlandet],True,62.408805,7.212250,combined,60,None,2024-02-04T07:34:19Z,7,6,4,6,https://fnugg.no//arenaoveroye/,Radiant sun
1,Fulufjellet Alpinsenter,[Østlandet],True,61.390801,12.746394,combined,40,None,2024-02-04T07:38:56Z,10,10,2,10,https://fnugg.no//fulufjellet/,Cloudy
2,Feforbakken,[Østlandet],True,61.538611,9.642069,combined,60,None,2024-02-04T07:38:58Z,7,6,1,6,https://fnugg.no//feforbakken/,Cloudy
3,Hornindal Skisenter,[Nord-Vestlandet],True,62.007834,6.561756,combined,80,None,2024-02-04T07:38:57Z,7,3,2,3,https://fnugg.no//hornindal/,Radiant sun
4,Gausta Skisenter,[Østlandet],True,59.880152,8.734026,combined,100,None,2024-02-04T15:29:55Z,42,15,13,15,https://fnugg.no//gausta/,Cloudy


In [390]:
cwd = Path.cwd()
file_path = cwd / 'dashboard' / 'ski-resorts.csv'

In [391]:
df_dashboard.to_csv(file_path, index=False)

In [392]:
resort_df = pd.read_csv(file_path)

In [393]:
resort_df.dtypes

Name                    object
Region                  object
Resort Open               bool
Latitude               float64
Longitude              float64
Weather Zone Type       object
Snow on Top (cm)       float64
Snow at Bottom (cm)    float64
Last Updated            object
Total Slopes             int64
Slopes Open              int64
Total Lifts              int64
Lifts Open               int64
Url                     object
Weekend Forecast        object
dtype: object

In [402]:
set([resort_df['Weekend Forecast'][0]])

{'Radiant sun'}

In [412]:
list = resort_df['Weekend Forecast'].unique().tolist()

In [421]:
sorted(list)

['Cloudy', 'Mostly sun', 'Radiant sun']